Matisse Colombon S4119576

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gsarti/ik-nlp-tutorials/blob/main/notebooks/W5E_Inseq_Analysis.ipynb)

In [1]:
# Run in Colab to install local packages
!pip install inseq

DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


finishing touches W2E# Exercise 1: Analyzing language generation models with Inseq 🐛

*Adapted in part from the [Inseq documentation](https://inseq.readthedocs.io/)*

Inseq is a toolkit based on the 🤗 Transformers and [Captum](https://captum.ai/docs/introduction) libraries for interpreting language generation models using feature attribution methods. Inseq allows you to analyze the behavior of a language generation model by computing the importance of each input token for each token in the generated output. The importance can be obtained using approaches based on attention, gradients and more, which we will see in more detail in the final lecture.

Inseq is a relatively new library, and it is still under active development (contributions welcome! 🙂). You can refer to the [Inseq paper](https://arxiv.org/abs/2302.13942) for an overview of the tool and some examples, or [this paper](https://www.semanticscholar.org/paper/Are-Character-level-Translations-Worth-the-Wait-An-Edman-Toral/ed7b51e4a5c4835218f6697b280afb2849211939) for a recent work from our GroNLP group on using Inseq to analyze character-level translation models.

In the following sections two simple use-cases of Inseq are presented.

## Attributing (Un)constrained Machine Translation

In this section we will use Inseq to compute the importance of each input token for each token in the generated output. We will use the [Helsinki-NLP/opus-mt-en-nl](https://huggingface.co/Helsinki-NLP/opus-mt-en-nl) model, which is a pretrained machine translation model from English to Dutch.

In [2]:
import inseq
import warnings

warnings.filterwarnings("ignore")

model = inseq.load_model("Helsinki-NLP/opus-mt-en-nl", "input_x_gradient")
out = model.attribute(
    "Don't get hot-headed mate, it's easy peasy to study models with Inseq!",
    attribute_target=True,
    step_scores=["probability"]
)
out.show()

Attributing with input_x_gradient...: 100%|██████████| 20/20 [00:00<00:00, 36.57it/s]


,▁Maak,▁je,▁niet,▁zo,▁druk,",",▁het,▁is,▁makkelijk,▁om,▁modellen,▁te,▁bestuderen,▁met,▁In,se,q,!,</s>
▁Don,0.045,0.036,0.02,0.028,0.022,0.026,0.02,0.014,0.02,0.01,0.011,0.012,0.016,0.012,0.007,0.006,0.005,0.027,0.02
',0.019,0.017,0.013,0.017,0.011,0.016,0.012,0.007,0.016,0.007,0.007,0.008,0.011,0.006,0.005,0.004,0.003,0.025,0.017
t,0.031,0.025,0.028,0.021,0.017,0.02,0.018,0.01,0.009,0.005,0.006,0.007,0.008,0.006,0.008,0.004,0.002,0.014,0.012
▁get,0.118,0.068,0.056,0.05,0.049,0.027,0.03,0.02,0.014,0.01,0.013,0.012,0.015,0.012,0.01,0.005,0.004,0.018,0.021
▁hot,0.066,0.073,0.071,0.088,0.084,0.042,0.026,0.022,0.016,0.01,0.014,0.011,0.012,0.011,0.009,0.008,0.007,0.022,0.019
-,0.021,0.02,0.024,0.027,0.025,0.018,0.01,0.007,0.008,0.005,0.005,0.005,0.007,0.005,0.005,0.005,0.006,0.013,0.009
headed,0.143,0.146,0.179,0.136,0.178,0.133,0.068,0.051,0.029,0.023,0.034,0.027,0.047,0.032,0.021,0.013,0.009,0.041,0.042
▁mate,0.085,0.072,0.081,0.061,0.07,0.095,0.052,0.02,0.02,0.013,0.014,0.013,0.02,0.015,0.01,0.005,0.005,0.021,0.022
",",0.02,0.02,0.017,0.016,0.015,0.03,0.019,0.013,0.014,0.006,0.005,0.007,0.008,0.006,0.004,0.004,0.003,0.021,0.018
▁it,0.017,0.012,0.012,0.01,0.009,0.021,0.026,0.022,0.014,0.012,0.008,0.013,0.014,0.009,0.005,0.004,0.003,0.017,0.014


<details>
    <summary> Observations </summary>

- The model translates idiomatic expressions in a non-literal way. Attribution scores reflect that the model is attributing strong importance to pieces of the idiom when translating (e.g. `_headed`, `_peas`), while also accounting for the prefix when producing an idiomatic translation (e.g. `_Maak`).
- The model gets progressively more confident in its translation as it generates more tokens. The first generated token is very unlikely.
</details>

Let's try now to **constrain** the generation to a more literal translation of the input. Attributing a prespecified output can be intuitively thought as a way to ask a model to justify a possible prediction. Note that this should be done with care, since if the output is very unlikely the results will be very noisy.

In [3]:
import inseq

model = inseq.load_model("Helsinki-NLP/opus-mt-en-nl", "input_x_gradient")
out = model.attribute(
    "Don't get hot-headed mate, it's easy peasy to study models with Inseq!",
    #"Niet heethoofdig worden maatje, het is gemakkelijk peasy om modellen te bestuderen met Inseq!",
    "Niet heethoofdig worden man, het is een makkie om modellen te bestuderen met Inseq!",
    attribute_target=True,
    step_scores=["probability"]
)
out.show()

Attributing with input_x_gradient...: 100%|██████████| 24/24 [00:00<00:00, 37.62it/s]


,▁Niet,▁heet,hoofd,ig,▁worden,▁man,",",▁het,▁is,▁een,▁,mak,kie,▁om,▁modellen,▁te,▁bestuderen,▁met,▁In,se,q,!,</s>
▁Don,0.08,0.041,0.013,0.018,0.036,0.024,0.028,0.017,0.011,0.013,0.009,0.011,0.006,0.009,0.01,0.013,0.016,0.011,0.008,0.006,0.005,0.027,0.013
',0.031,0.025,0.012,0.015,0.024,0.016,0.024,0.015,0.006,0.008,0.005,0.006,0.004,0.007,0.007,0.008,0.011,0.006,0.004,0.004,0.003,0.026,0.01
t,0.045,0.021,0.013,0.013,0.024,0.02,0.018,0.011,0.007,0.007,0.006,0.005,0.003,0.005,0.006,0.007,0.008,0.005,0.007,0.004,0.002,0.014,0.009
▁get,0.095,0.041,0.018,0.034,0.071,0.042,0.025,0.019,0.013,0.011,0.009,0.011,0.006,0.008,0.011,0.01,0.015,0.01,0.01,0.005,0.004,0.02,0.021
▁hot,0.082,0.091,0.09,0.055,0.064,0.042,0.026,0.021,0.016,0.014,0.013,0.015,0.01,0.012,0.011,0.01,0.012,0.01,0.009,0.007,0.007,0.023,0.017
-,0.023,0.023,0.032,0.019,0.018,0.021,0.013,0.01,0.006,0.007,0.005,0.006,0.003,0.004,0.005,0.005,0.007,0.005,0.005,0.005,0.006,0.012,0.007
headed,0.126,0.186,0.263,0.143,0.092,0.145,0.071,0.06,0.039,0.032,0.033,0.04,0.02,0.026,0.032,0.027,0.046,0.03,0.022,0.013,0.009,0.041,0.039
▁mate,0.08,0.05,0.056,0.112,0.071,0.197,0.066,0.088,0.042,0.04,0.024,0.022,0.016,0.021,0.036,0.02,0.02,0.025,0.021,0.006,0.005,0.028,0.04
",",0.025,0.021,0.015,0.022,0.018,0.026,0.021,0.014,0.012,0.009,0.006,0.008,0.005,0.007,0.006,0.007,0.008,0.006,0.004,0.004,0.003,0.021,0.011
▁it,0.02,0.016,0.012,0.012,0.011,0.016,0.019,0.017,0.024,0.013,0.011,0.011,0.005,0.01,0.008,0.014,0.014,0.008,0.005,0.004,0.003,0.017,0.009


### Your turn to comment

Comment on the resulting scores from the constrained, less idiomatic example, putting them in relation to the unconstrained, more idiomatic one. Consider the following aspects, but feel free to explore other examples and add your own observations:
1. How are importance scores distributed on idiomatic and non-idiomatic tokens?
2. What is the difference in probability between the two examples?
3. Do you notice some patterns regarding the low-probability tokens in the second example?
4. When is the target prefix playing a more important role in the generation, according to the attribution scores?

1: On idiomatic translations the model places more importance on the whole idiom while, while on non idiomic tokens the importance is more focussed on one token.

2: In the literal translation it is pretty sure about tokens hoofd, ig and worden (~0.7) however not very sure about the other tokens. In the idiomatic translation it is per token less sure but the probabilities are more steady at about 0.45.

3: The low probabilities seem to be for the literal translation parts, like the 'heet' from 'hot-headed' has a low probability.

4: As you can see in the Target salliancy heatmap the prefix from the first sentence (Maak) is more important then the prefix from the second sentence (Niet). So the Target prefix is more important in the idiomical translation.

## Contrastive Attribution for Motivating Preferences

In the previous section we used importance scores produced by attributing next token’s probability, which can be seen as answering the question “Which elements of the input are the most relevant to produce the next generation step?”.

However, in many cases we might be more interested in understanding why our model generated its output **rather than another one that we consider to be more likely**. The paper [“Interpreting Language Models with Contrastive Explanations”](https://arxiv.org/abs/2202.10419) by Yin and Neubig (2022) proposes a contrastive attribution method that can be used to answer this question. The method is integrated in Inseq and can be used as follows:

In [4]:
import inseq

model = inseq.load_model("google/flan-t5-base", "input_x_gradient")

out = model.attribute(
    "Does 3 + 3 equal 6?",
    # Fix the original target
    "yes",
    attributed_fn="contrast_prob_diff",
    # Also show the probability delta between the two options
    step_scores=["contrast_prob_diff", "probability"],
    contrast_targets="no",
)

# Normally attributions are normalized to sum up to 1
# Here we want to see how they contribute to the probability difference
out.show()

Attributing with input_x_gradient...: 100%|██████████| 3/3 [00:00<00:00,  8.69it/s]


,▁no → ▁yes,</s>
▁Does,0.149,0.0
▁3,0.09,0.0
▁+,0.089,0.0
▁3,0.097,0.0
▁equal,0.191,0.0
▁6,0.154,0.0
?,0.11,0.0
</s>,0.12,0.0
contrast_prob_diff,0.05,-0.0
probability,0.523,1.0


We can see that the model is relying more heavily on formulation keywords (`Does`, `equal`) and less on the numbers to determine the answer. The gap between the positive and negative answer is also quite small (5%), suggesting that the model is not very confident in its answer. Changing the input to `Does 3 + 3 equal 7?` confirms that the actual expression is not playing a relevant role in the generation.

> ⚠️ **Important**: Since contrastive attribution compares the probabilities of a pair of `(original, contrastive)` tokens, in order for it to work, the compared sequences must have the same length. For example, if "yes" was tokenized as `_y`, `es` we couldn't have compared it with a single token `_no` using Inseq.

### Your turn to attribute

Using the generation model and a task of your choice, try to use contrastive attribution on at least three examples to highlight some interesting pattern of your choice. We encourage you to explore whathever you find most interesting, but here are some suggestions:

- Is negation relevant in producing the correct answer in open question answering models like the one we used in the previous example?

- When translating sentences like `The nurse went to the hospital` to a gendered language like Spanish, Italian or German, the model will have to select a gender for the subject. What is the model relying on to make this choice?

- Considering a fixed example like `Does 3 + 3 = 6?` but using models with increasingly more parameters (e.g. `flan-t5-small`, `flan-t5-base`, `flan-t5-large`), how does input importance and model confidence change?

After producing the visualizations, comment on the results and try to explain what you observe.

Inseq also supports attribution of quantized models (see [example](https://inseq.readthedocs.io/examples/locate_gpt2_knowledge.html)), in case you want to explore using larger models for your analysis. Refer to the [Inseq documentation](https://inseq.readthedocs.io/) for more details.

I looked at how different size models affect the importance of tokens and confidence rates.
First looking at the probabilities we see that the how bigger the model gets how more confident it is on the answer No. 0.61 to 0.838 to 0.905, the biggest jump is from small to base, the jump from base to large is smaller but still clearly exists.

In all models the tokens London, Capital and France have the highest attribution, which seem like the correct words to answer the question. 

In [6]:
import inseq

models = {
    "small": inseq.load_model("google/flan-t5-small", "input_x_gradient"),
    "base": inseq.load_model("google/flan-t5-base", "input_x_gradient"),
    "large": inseq.load_model("google/flan-t5-large", "input_x_gradient")
}

question = "Is the London the capital of France?"

for size, model in models.items():

    
    out = model.attribute(
        question,
        # Fix the original target
        "no",
        attributed_fn="contrast_prob_diff",
        # Also show the probability delta between the two options
        step_scores=["contrast_prob_diff", "probability"],
        contrast_targets="yes",
    )
    
    out.weight_attributions("contrast_prob_diff")
    print(f"Model: {size}")
    out.show()

Attributing with input_x_gradient...: 100%|██████████| 3/3 [00:00<00:00, 18.09it/s]

Model: small


,▁yes → ▁no,</s>
▁I,0.028,-0.0
s,0.023,-0.0
▁the,0.013,-0.0
▁London,0.045,-0.0
▁the,0.017,-0.0
▁capital,0.041,-0.0
▁of,0.011,-0.0
▁France,0.037,-0.0
?,0.025,-0.0
</s>,0.028,-0.0


Attributing with input_x_gradient...: 100%|██████████| 3/3 [00:00<00:00, 12.29it/s]

Model: base


,▁yes → ▁no,</s>
▁I,0.062,0.0
s,0.05,0.0
▁the,0.05,0.0
▁London,0.126,0.0
▁the,0.058,0.0
▁capital,0.084,0.0
▁of,0.031,0.0
▁France,0.09,0.0
?,0.082,0.0
</s>,0.064,0.0


Attributing with input_x_gradient...: 100%|██████████| 3/3 [00:00<00:00,  6.11it/s]

Model: large


,▁yes → ▁no,</s>
▁I,0.087,-0.0
s,0.047,-0.0
▁the,0.108,-0.0
▁London,0.149,-0.0
▁the,0.069,-0.0
▁capital,0.117,-0.0
▁of,0.031,-0.0
▁France,0.099,-0.0
?,0.056,-0.0
</s>,0.122,-0.0
